<a href="https://colab.research.google.com/github/dlotw/Algorithms/blob/master/MTH9821_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import random
import math
from scipy.stats import norm

In [2]:
# question 1.1
# parameters
a0 = 2.50662823884
b0 = -8.47351093090
a1 = -18.61500062529
b1 = 23.08336743743
a2 = 41.39119773534
b2 = -21.06224101826
a3 = -25.44106049637
b3 = 3.13082909833
c0 = 0.3374754822726147
c5 = 0.0003951896511919
c1 = 0.9761690190917186
c6 = 0.0000321767881768
c2 = 0.1607979714918209
c7 = 0.0000002888167364
c3 = 0.0276438810333863
c8 = 0.0000003960315187
c4 = 0.0038405729373609

# Beasley–Springer–Moro algorithm function
def bsm(u: float):
    """
    intput: u is between 0 and 1
    output: x, approximation to \phi^-1(u)
    """
    y = u - 0.5
    if abs(y) < 0.42:
        r = y * y
        x = y * (((a3 * r + a2) * r + a1) * r + a0) / ((((b3 * r + b2) * r + b1) * r + b0) * r + 1)
    else:
        r = u
        if y > 0: r = 1 - u
        r = math.log(-math.log(r))
        x = c0 + r * (c1 + r * (c2 + r * (c3 + r * (c4+ r * (c5 + r * (c6 + r * (c7 + r * c8)))))))
        if y < 0:
            x = -x
    return x


In [3]:
# question 1.2
def black_scholes(S, K, T, r, q, sigma, option_type):

    d1 = (np.log(S / K) + (r - q + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        option_price = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        delta = np.exp(-q * T) * norm.cdf(d1)
    elif option_type == 'put':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)
        delta = -np.exp(-q * T) * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

    gamma = np.exp(-q * T) * norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)

    return option_price, delta, gamma, vega

In [4]:
# question 1.4
def calculate_option_price(S0, K, T, r, q, sigma, delta_s, num_simulations):
    np.random.seed(42)
    z = np.random.normal(0, 1, num_simulations)
    # stock price
    ST = S0 * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)
    ST_plus = (S0 + delta_s) * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)
    ST_minus = (S0 - delta_s) * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)
    # option price
    option_payoff = np.exp(-r * T) * (ST - K)
    option_payoff[option_payoff < 0] = 0
    option_price = np.mean(option_payoff)
    option_payoff_plus = np.exp(-r * T) * (ST_plus - K)
    option_payoff_plus[option_payoff_plus < 0] = 0
    option_price_plus = np.mean(option_payoff_plus)
    option_payoff_minus = np.exp(-r * T) * (ST_minus - K)
    option_payoff_minus[option_payoff_minus < 0] = 0
    option_price_minus = np.mean(option_payoff_minus)
    return option_price, option_price_plus, option_price_minus

# parameters
S0 = 100  # Current stock price
K = 100  # Strike price
T = 1    # Time to expiration (in years)
r = 0.05 # Risk-free interest rate
sigma = 0.2 # Volatility
q = 0.02  # Dividend yield

# bs value and delta
bsm_value, bsm_delta, bsm_gamma, bsm_vega = black_scholes(S0, K, T, r, q, sigma, "call")
print("BSM value:")
print(bsm_value, bsm_delta, bsm_gamma)

delta_s = 0.1 # delta_s

result_df = pd.DataFrame(columns=['Delta', 'Delta_diff', 'Gamma', 'Gamma_diff'])
for k in range(10):
    N = 10000 * (2**k)
    # option price
    option_price, option_price_plus, option_price_minus = calculate_option_price(S0, K, T, r, q, sigma, delta_s, N)
    # delta calculation
    delta = (option_price_plus - option_price_minus) / (2 * delta_s)
    # gamma calculation
    gamma = (option_price_plus - 2 * option_price + option_price_minus) / (delta_s)**2
    # diff from bsm value
    delta_diff = N**0.5 * abs(delta - bsm_delta)
    gamma_diff = N**0.5 * abs(gamma - bsm_gamma)
    # add to result_df

    result_df = result_df.append({'Delta': delta, 'Delta_diff': delta_diff, 'Gamma': gamma, 'Gamma_diff': gamma_diff}, ignore_index=True)

print("Monte Carlo value:")
print(result_df)

BSM value:
9.227005508154036 0.586851146134764 0.018950578755008718


<ipython-input-4-b75609cb2964>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Delta': delta, 'Delta_diff': delta_diff, 'Gamma': gamma, 'Gamma_diff': gamma_diff}, ignore_index=True)
<ipython-input-4-b75609cb2964>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Delta': delta, 'Delta_diff': delta_diff, 'Gamma': gamma, 'Gamma_diff': gamma_diff}, ignore_index=True)
<ipython-input-4-b75609cb2964>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Delta': delta, 'Delta_diff': delta_diff, 'Gamma': gamma, 'Gamma_diff': gamma_diff}, ignore_index=True)
<ipython-input-4-b75609cb2964>:50: FutureWarning: The frame.append method is deprecated 

Monte Carlo value:
      Delta  Delta_diff     Gamma  Gamma_diff
0  0.586687    0.016438  0.020190    0.123937
1  0.590236    0.478761  0.018756    0.027557
2  0.586199    0.130472  0.020305    0.270880
3  0.588093    0.351273  0.020660    0.483445
4  0.587782    0.372379  0.020448    0.599046
5  0.586239    0.346067  0.019418    0.264689
6  0.586397    0.363274  0.019043    0.074077
7  0.586354    0.562058  0.019098    0.167173
8  0.586890    0.061634  0.019086    0.216262
9  0.586879    0.062573  0.019060    0.247508


<ipython-input-4-b75609cb2964>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Delta': delta, 'Delta_diff': delta_diff, 'Gamma': gamma, 'Gamma_diff': gamma_diff}, ignore_index=True)


In [5]:
# Q3.1

S0 = 50
K = 55
T = 0.5
r = 0.04
q = 0
sigma = 0.3

bs_value, bs_delta, bs_gamma, bs_vega = black_scholes(S0, K, T, r, q, sigma, "put")
print(f"B-S value: {bs_value}")

B-S value: 6.6166546586411705


In [6]:
# Q3.2

def linear_congruential_generator(N: int):
    """
    Generates N independent samples from uniform distribution on [0,1]
    """

    # Parameters for the Linear Congruential Generator.
    a = 39373
    c = 0
    k = 2**31 - 1

    samples = np.zeros(N)
    xi = 1

    for i in range(N):
        xi = (a * xi + c) % k
        ui = xi / k
        samples[i] = ui

    return samples

def simulated_spot(S0, T, r, q, sigma, zi):
    return S0 * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * zi)

def put_payoff(Si, K, T, r):
    return np.exp(-r * T) * np.maximum(K - Si, 0)

In [7]:
# inverse transform

a0 = 2.50662823884
b0 = -8.47351093090
a1 = -18.61500062529
b1 = 23.08336743743
a2 = 41.39119773534
b2 = -21.06224101826
a3 = -25.44106049637
b3 = 3.13082909833
c0 = 0.3374754822726147
c5 = 0.0003951896511919
c1 = 0.9761690190917186
c6 = 0.0000321767881768
c2 = 0.1607979714918209
c7 = 0.0000002888167364
c3 = 0.0276438810333863
c8 = 0.0000003960315187
c4 = 0.0038405729373609

# Beasley–Springer–Moro algorithm function
def inverse_transform(u: float):
    """
    Generate independent samples from standard normal distribution by using
    independent uniform samples with inverse transform method
    """
    y = u - 0.5
    if abs(y) < 0.42:
        r = y * y
        x = y * (((a3 * r + a2) * r + a1) * r + a0) / ((((b3 * r + b2) * r + b1) * r + b0) * r + 1)
    else:
        r = u
        if y > 0: r = 1 - u
        r = math.log(-math.log(r))
        x = c0 + r * (c1 + r * (c2 + r * (c3 + r * (c4+ r * (c5 + r * (c6 + r * (c7 + r * c8)))))))
        if y < 0:
            x = -x
    return x

In [8]:
result_df_it = pd.DataFrame(columns=['N', 'Price', 'ERR'])

for k in range(10):
    price = 0
    N = 10000 * (2**k)
    U = linear_congruential_generator(N)
    for ui in U:
        zi = inverse_transform(ui)
        ST = simulated_spot(S0, T, r, q, sigma, zi)
        p = put_payoff(ST, K, T, r)
        price += p
    price /= N
    err = np.abs(bs_value - price)

    result_df_it = result_df_it.append({'N': N, 'Price': price, 'ERR': err}, ignore_index=True)

print(result_df_it)

<ipython-input-8-2bff1c1749a5>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_it = result_df_it.append({'N': N, 'Price': price, 'ERR': err}, ignore_index=True)
<ipython-input-8-2bff1c1749a5>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_it = result_df_it.append({'N': N, 'Price': price, 'ERR': err}, ignore_index=True)
<ipython-input-8-2bff1c1749a5>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_it = result_df_it.append({'N': N, 'Price': price, 'ERR': err}, ignore_index=True)
<ipython-input-8-2bff1c1749a5>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_it = result_df_it.append({'N': N, 

           N     Price       ERR
0    10000.0  6.600967  0.015688
1    20000.0  6.650060  0.033405
2    40000.0  6.628623  0.011969
3    80000.0  6.631274  0.014619
4   160000.0  6.637338  0.020683
5   320000.0  6.624188  0.007533
6   640000.0  6.621166  0.004511
7  1280000.0  6.616959  0.000304
8  2560000.0  6.620114  0.003459
9  5120000.0  6.617664  0.001009


<ipython-input-8-2bff1c1749a5>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_it = result_df_it.append({'N': N, 'Price': price, 'ERR': err}, ignore_index=True)


In [9]:
# acceptance-rejection
import numpy as np

def acceptance_rejection(u1: float, u2: float, u3: float):
    """
    Generate independent samples from standard normal distribution by using
    acceptance-rejection methods
    """
    X = -np.log(u1) if u3 < 0.5 else np.log(u1)

    # Calculate the density of the standard normal distribution at X
    f_X = (1/np.sqrt(2*np.pi)) * np.exp(-0.5 * X**2)

    # Calculate the density of the symmetrized exponential distribution at X
    g_X = 0.5 * np.exp(-np.abs(X))

    # Calculate the acceptance probability
    c = np.sqrt(2*np.e/np.pi)
    accept_prob = f_X / (c * g_X)

    # Accept or reject the sample based on the acceptance probability
    return X if u2 < accept_prob else None

In [10]:
result_df_ar = pd.DataFrame(columns=['N', 'N_AR', 'Price', 'ERR'])

for k in range(10):
    price = 0
    N_AR = 0
    N = 10000 * (2**k)
    U = linear_congruential_generator(N)

    for i in range(int(N//3)):
        u1 = U[3*i]
        u2 = U[3*i+1]
        u3 = U[3*i+2]
        zi = acceptance_rejection(u1,u2,u3)
        if zi != None:
            N_AR += 1
            ST = simulated_spot(S0, T, r, q, sigma, zi)
            p = put_payoff(ST, K, T, r)
            price += p

    price /= N_AR
    err = np.abs(bs_value - price)

    result_df_ar = result_df_ar.append({'N': N, 'N_AR': N_AR, 'Price': price, 'ERR': err}, ignore_index=True)

print(result_df_ar)

<ipython-input-10-f792f8755a4a>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_ar = result_df_ar.append({'N': N, 'N_AR': N_AR, 'Price': price, 'ERR': err}, ignore_index=True)
<ipython-input-10-f792f8755a4a>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_ar = result_df_ar.append({'N': N, 'N_AR': N_AR, 'Price': price, 'ERR': err}, ignore_index=True)
<ipython-input-10-f792f8755a4a>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_ar = result_df_ar.append({'N': N, 'N_AR': N_AR, 'Price': price, 'ERR': err}, ignore_index=True)
<ipython-input-10-f792f8755a4a>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


           N       N_AR     Price       ERR
0    10000.0     2546.0  6.528255  0.088400
1    20000.0     5080.0  6.663936  0.047281
2    40000.0    10161.0  6.612578  0.004076
3    80000.0    20269.0  6.646650  0.029995
4   160000.0    40502.0  6.585181  0.031473
5   320000.0    81117.0  6.643441  0.026786
6   640000.0   162220.0  6.606467  0.010188
7  1280000.0   324352.0  6.607645  0.009009
8  2560000.0   648827.0  6.609537  0.007118
9  5120000.0  1297408.0  6.616177  0.000478


<ipython-input-10-f792f8755a4a>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_ar = result_df_ar.append({'N': N, 'N_AR': N_AR, 'Price': price, 'ERR': err}, ignore_index=True)


In [11]:
# box-muller (marsaglia-bray)

def box_muller(u1: float, u2: float):
    """
    Generate independent samples from standard normal distribution by using
    box-muller methods
    """
    v1 = 2 * u1 - 1
    v2 = 2 * u2 - 1
    X = v1**2 + v2**2

    # If X is outside the unit circle, start over
    if X >= 1 or X == 0:
        return

    multiplier = np.sqrt(-2 * np.log(X) / X)

    z1 = v1 * multiplier
    z2 = v2 * multiplier

    return z1, z2

In [12]:
result_df_bm = pd.DataFrame(columns=['N', 'N_BM', 'Price', 'ERR'])

for k in range(10):
    price = 0
    N_BM = 0
    N = 10000 * (2**k)
    U = linear_congruential_generator(N)

    for i in range(int(N//2)):
        u1 = U[i]
        u2 = U[2*i]
        z = box_muller(u1,u2)
        if z != None:
            N_BM += 2
            for zi in z:
                ST = simulated_spot(S0, T, r, q, sigma, zi)
                p = put_payoff(ST, K, T, r)
                price += p

    price /= N_BM
    err = np.abs(bs_value - price)

    result_df_bm = result_df_bm.append({'N': N, 'N_BM': N_BM, 'Price': price, 'ERR': err}, ignore_index=True)

print(result_df_bm)

<ipython-input-12-a67a8231fa65>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_bm = result_df_bm.append({'N': N, 'N_BM': N_BM, 'Price': price, 'ERR': err}, ignore_index=True)
<ipython-input-12-a67a8231fa65>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_bm = result_df_bm.append({'N': N, 'N_BM': N_BM, 'Price': price, 'ERR': err}, ignore_index=True)
<ipython-input-12-a67a8231fa65>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_bm = result_df_bm.append({'N': N, 'N_BM': N_BM, 'Price': price, 'ERR': err}, ignore_index=True)
<ipython-input-12-a67a8231fa65>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


           N       N_BM     Price       ERR
0    10000.0     7826.0  6.549263  0.067392
1    20000.0    15626.0  6.582472  0.034182
2    40000.0    31430.0  6.607572  0.009083
3    80000.0    62944.0  6.632364  0.015709
4   160000.0   126078.0  6.624103  0.007448
5   320000.0   251454.0  6.620529  0.003874
6   640000.0   502592.0  6.617175  0.000521
7  1280000.0  1005342.0  6.616369  0.000286
8  2560000.0  2010980.0  6.619829  0.003174
9  5120000.0  4022982.0  6.621176  0.004522


<ipython-input-12-a67a8231fa65>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df_bm = result_df_bm.append({'N': N, 'N_BM': N_BM, 'Price': price, 'ERR': err}, ignore_index=True)
